In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

In [6]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [7]:
df['text'][0]

"there were nights where i was sure   i wouldn't see the morning sun   and there were days that seemed so dark   i couldn't wait for night to come      i couldn't stand to think about how   my life used to be   and how without a single warning   it all slipped away from me      and like a fool i thought that i could fight   the shadows on my own   to the dark i was no stranger   but this was stronger than i'd known      and by the time i knew that i was in too deep   i'd gone too far   and the light that used to guide me   had faded from my heart      and i found myself in places i thought i'd never go   surrounded by strangers i was so far from home   and i don't know how you found me   all i know is i owe you everything to you   yes i do      [chorus]   and i thank you for my heart   i thank you for my life   and i thank god for grace and mercy   and that you became my wife   i'm seeing for the first time   the stars, the sun and moon   but they've got nothing on the power   of this 

In [8]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [9]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [16]:
df['song'][0]

'Thank You'

In [14]:
similarity[0]

array([1.        , 0.08709491, 0.01558478, ..., 0.11299213, 0.05717278,
       0.06513018])

df['song'][0]

In [17]:
df[df['song']=='Thank You'].index[0]

0

# recommedation function

In [18]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Thank You')

['I Am Yours',
 'Thank You',
 'Make It Last Forever',
 'Thank You For The Music',
 'Thank God For The Bomb',
 'Sunshine On Leith',
 'Days Like These',
 'Behold A Lady',
 'Song For Noa',
 'Here',
 'Love Has Got The Power',
 'Before I Break',
 'If You Were Me',
 'A Healer Like Time',
 'Could I Ever',
 'Claim My Place (Interlude)',
 'A Secret Love',
 'Country Home',
 'Life Can Be So Beautiful',
 'The Longest Pee']

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))